In [1]:
%load_ext autoreload
%autoreload 2
import sys, os, torch, datetime, time
import numpy as np
sys.path.append("..")
import rfi_ml
import matplotlib.pyplot as plt

In [2]:
Ntrain = 100000 #the size of the training set
Ntest = 10
sigma_low = 20
sigma_high = 50
Nepochs = 30
Nrfi = 1
#Np = 2**13 #8k
#hidden_dim = 2**11
Np = 2**11 #2k
hidden_dim = 2**10
z_dim = 16
Nfft = Np // 2 + 1
#Pk = np.ones(Nfft)
Pk = None
#lamb = 0.9 #lambda, scaling down Gaussianized amplitude
np.random.seed(0) #for debugging
torch.manual_seed(0) #more debugging
start_time = time.time()

t = rfi_ml.ToyGenerator(Np=Np, Pk=Pk)
g = torch.stack([torch.from_numpy(t.getGaussian()) for i in range(Ntrain+Ntest)])
rfi_array = np.zeros((Ntrain+Ntest, Np), dtype=float)
for i in range(Ntrain + Ntest):
    rfi_timestream = np.zeros(Np)
    for j in range(Nrfi):
        rfi_timestream += t.getNonGaussianLocalized(sigma=(sigma_low, sigma_high)) 
        #rfi_timestream += t.getNonGaussianLocalized(sigma=(sigma_low, sigma_high), ampl=(0.05, 0.05)) 
        #rfi_timestream += t.getNonGaussianNonlocalized(freq = ((2*np.pi)/(Np), (4*np.pi)/Np), Pflip=0.0) 
    rfi_array[i,:] += rfi_timestream
ng = torch.from_numpy(rfi_array)
#ng = torch.stack([torch.from_numpy(t.getNonGaussianLocalized(sigma=(sigma_low, sigma_high))) for i in range(Ntrain+Ntest)])


In [3]:
n = rfi_ml.RFIDetect(Np, Nepochs = Nepochs, z_dim = z_dim, hidden_dim = hidden_dim)

In [4]:
n.train(g[:Ntrain], ng[:Ntrain])

[  0/30][  0/3125]	Loss: 0.0012586177
[  0/30][100/3125]	Loss: 0.0007741635
[  0/30][200/3125]	Loss: 0.0007200703
[  0/30][300/3125]	Loss: 0.0006540589
[  0/30][400/3125]	Loss: 0.0007103161
[  0/30][500/3125]	Loss: 0.0006933152
[  0/30][600/3125]	Loss: 0.0007328665
[  0/30][700/3125]	Loss: 0.0007115927
[  0/30][800/3125]	Loss: 0.0007171914
[  0/30][900/3125]	Loss: 0.0007468255
[  0/30][1000/3125]	Loss: 0.0006828384
[  0/30][1100/3125]	Loss: 0.0007049154
[  0/30][1200/3125]	Loss: 0.0007050228
[  0/30][1300/3125]	Loss: 0.0007271452
[  0/30][1400/3125]	Loss: 0.0007215503
[  0/30][1500/3125]	Loss: 0.0006983735
[  0/30][1600/3125]	Loss: 0.0007259531
[  0/30][1700/3125]	Loss: 0.0007092034
[  0/30][1800/3125]	Loss: 0.0007650827
[  0/30][1900/3125]	Loss: 0.0006984577
[  0/30][2000/3125]	Loss: 0.0006671052
[  0/30][2100/3125]	Loss: 0.0006751818
[  0/30][2200/3125]	Loss: 0.0006421293
[  0/30][2300/3125]	Loss: 0.0006695918
[  0/30][2400/3125]	Loss: 0.0006911795
[  0/30][2500/3125]	Loss: 0.0006455

In [5]:
start_test = time.time()
testout = n.evaluate(g[Ntrain:], ng[Ntrain:])
end_test = time.time()

Epochs:  30
N tests:  10
Length of timestream:  2048
Timestream Obs-Expect RMS:  [0.00391986 0.00352932 0.00415077 0.00327644 0.00385222 0.00415577
 0.00491384 0.0036613  0.00322086 0.00434949]
Avg RMS for all tests:  0.003902986647196114 

Fraction of RFI power remaining:  [0.05032707 0.05204281 0.04453969 0.0438194  0.05476431 0.04633184
 0.05922997 0.09520987 0.03903036 0.0390275 ]
Remainder as fraction of signal power:  [0.03990329 0.03225859 0.04701063 0.03153598 0.0405046  0.04855021
 0.07038621 0.04116149 0.03035761 0.05774853]
Avg of RFI power remaining:  0.05243228272255893
Avg amp of Gaussian signal:  0.015037855787992983


In [6]:
n.plot_eval(testout, g[Ntrain:], ng[Ntrain:])

Saving file...rfi_ml/2021-09-30_23-51-49_epoch_0000030_test_0.png
Saving file...rfi_ml/2021-09-30_23-51-49_overplot_test_0.png
Saving file...rfi_ml/2021-09-30_23-51-49_epoch_0000030_test_1.png
Saving file...rfi_ml/2021-09-30_23-51-49_overplot_test_1.png
Saving file...rfi_ml/2021-09-30_23-51-49_epoch_0000030_test_2.png
Saving file...rfi_ml/2021-09-30_23-51-49_overplot_test_2.png
Saving file...rfi_ml/2021-09-30_23-51-49_epoch_0000030_test_3.png
Saving file...rfi_ml/2021-09-30_23-51-49_overplot_test_3.png
Saving file...rfi_ml/2021-09-30_23-51-49_epoch_0000030_test_4.png
Saving file...rfi_ml/2021-09-30_23-51-49_overplot_test_4.png
Saving file...rfi_ml/2021-09-30_23-51-49_epoch_0000030_test_5.png
Saving file...rfi_ml/2021-09-30_23-51-49_overplot_test_5.png
Saving file...rfi_ml/2021-09-30_23-51-49_epoch_0000030_test_6.png
Saving file...rfi_ml/2021-09-30_23-51-49_overplot_test_6.png
Saving file...rfi_ml/2021-09-30_23-51-49_epoch_0000030_test_7.png
Saving file...rfi_ml/2021-09-30_23-51-49_over

In [7]:
print("Runtime: ", time.time()-start_time)
print("Evaltime: ", end_test-start_test)

Runtime:  1159.1178772449493
Evaltime:  0.29807043075561523
